# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

import re
import io
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as STOP_WORDS
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [ ]:
# load data from database
engine = create_engine('sqlite:///figure8data.db')
df = pd.read_sql_table('figure8data', engine)
X = df.iloc[:, 1]
Y = df.iloc[:, 4:]
Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    text = re.sub(r"[^A-Za-z]", " ", text).strip().lower()
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    clean_tokens = [stemmer.stem(tok).strip().lower() for tok in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
tfidf_transformer = TfidfVectorizer(lowercase=True, stop_words=set(STOP_WORDS.words("english")), tokenizer=tokenize, 
                                                max_df=0.75, min_df=1, max_features=100000, 
                                                analyzer='word')
svd_transformer = TruncatedSVD(n_components=200, n_iter=20, random_state=42)
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100), n_jobs=-1) 

pipeline = Pipeline([('tfidf_transformer', tfidf_transformer), ('SVD', svd_transformer), ('classifier', classifier)])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_df[column]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'tfidf_transformer__max_df': (0.5, 0.75),
    'tfidf_transformer__max_features': (None, 100000),
    'SVD__n_components': (50, 100, 200),
    'SVD__n_iter': (20 ,40),
    'classifier__estimator__n_estimators': (100, 200)}
cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, verbose=3)

sample_ids = np.random.randint(0, X.shape[0], int(.15 * X.shape[0]))
X_train, X_test, y_train, y_test = train_test_split(X.iloc[sample_ids], Y.iloc[sample_ids], test_size=.2)
cv.fit(X_train, y_train)

In [ ]:
# Train model with best params on full data
tfidf_transformer = TfidfVectorizer(lowercase=True, stop_words=set(STOP_WORDS.words("english")), tokenizer=tokenize, 
                                                max_df=0.75, min_df=1, max_features=None, 
                                                analyzer='word')
svd_transformer = TruncatedSVD(n_components=100, n_iter=50, random_state=42)
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100), n_jobs=-1) 

pipeline = Pipeline([('tfidf_transformer', tfidf_transformer), ('SVD', svd_transformer), ('classifier', classifier)])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2)
pipeline.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_df[column]))

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(classifier, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)